In [2]:
import pandas as pd
from stage import fullPrint, getIndustryStage, getStage
from datetime import datetime, timedelta
import numpy as np
import pickle
import os
import investpy

PEAK = 12
allList = []
industryList = pd.read_pickle("stockData/industryList.pkl")

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"
    }

i = 0
now = datetime.now()

In [2]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
WMA30SLOPE = 7
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,WMA30SLOPE]/df.iat[index-1,WMA30SLOPE]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
        
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])

In [3]:
spDf = investpy.get_index_historical_data(index='S&P 500', country='United States', from_date='01/01/2010', to_date='01/01/2022', as_json=False, order='ascending', interval='Weekly')
spDf = spDf[~spDf.index.duplicated()]
spDf['Volume'] = spDf['Volume'].replace(to_replace=0, method='ffill')
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf = spDf.drop(['Open','High','Low','Currency'],axis=1)
spDf.to_pickle("stockData/Spy.pkl")

Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

In [4]:
for industry in industryList:
    try:
        stockDf = pd.read_pickle('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
        if stockDf['Close'].isnull().values.any():
            os.remove('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
        stockDf['fiveYearHigh'] = stockDf['Close'].rolling(window=260).max().shift(1).fillna(0)
        stockDf['WMA30'] = stockDf['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['Volume'] = stockDf['Volume'].replace(to_replace=0, method='ffill')
        stockDf['volumePerc'] = stockDf['Volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        stockDf['ticker'] = str(industry[1])
        stockDf = stockDf.drop(['Open','High','Low','Currency'],axis=1)
        stockDf.to_pickle('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
    except:
        pass

('XLP', 'DJTobacco')


Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

('XLP', 'DJSoftDrinks')


Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

('XLY', 'DJApparelRetailers')


Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]